# Descarga datos esios

## Documentacion:

La nueva web pública de e·sios pone a disposición de todos los usuarios una API para la descarga de información, cuya documentación se encuentra disponible en https://api.esios.ree.es.

Para poder utilizar esta API deberán solicitar un token personal enviando un correo a **consultasios@ree.es**. En mi caso:

token: 9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10

La solicitud será para un periodo de tiempo, seleccionando los datos por código indicador:

https://api.esios.ree.es/indicator/getting_a_specific_indicator_filtering_values_by_a_date_range


## 1. Importamos librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
pd.options.display.max_columns= None

In [3]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary or bytes to be sent in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      /anaconda3/lib/python3.6/site-packages/requests/api.py
Type:      function


## 2. Definición features

Creamos diccionario

In [4]:
d={10010: 'P48_EOLICA',
   10027: 'P48_DEMANDA',
   10008: 'P48_CARBON',
   #10015: 'P48_SALDO_INTERC',
   475: 'POT_DISP_HULLA_ANT',  #Los tendremos que sumar para obtener MW disponibles Carbón
   476: 'POT_DISP_HULA_SB',
   600: [['PRECIO_MD_ESP',3],['PRECIO_MD_FR',2]],
   612: [['PRECIO_I1',3]],
   613: [['PRECIO_I2',3]]  #Si PI1=PMD, miramos PMI2
  }

**NOTA**:

He comentado el indicador 10015 en esta recopilación de datos históricos para crear el dataset *datos_esios.csv* porque durante el mes de mayo'16 contiene huecos que hacen que la función genérica arroje el siguiente error al utilizar la función concat:

    Error: Shape of passed values is (4, 26340), indices imply (4, 26331)

Por ello, para evitar problemas, los he tratado en este libro por separado. En el recopilatorio de funciones para poder actualizar el modelo en un futuro, lo introduciré con el resto de indicadores. 

In [5]:
d.items()

dict_items([(10010, 'P48_EOLICA'), (10027, 'P48_DEMANDA'), (10008, 'P48_CARBON'), (475, 'POT_DISP_HULLA_ANT'), (476, 'POT_DISP_HULA_SB'), (600, [['PRECIO_MD_ESP', 3], ['PRECIO_MD_FR', 2]]), (612, [['PRECIO_I1', 3]]), (613, [['PRECIO_I2', 3]])])

En el caso del id_precios habrá que adentrarse un nivel adicional y consultar GEOIDS:

* ESPAÑA = 3
* FRANCIA = 2

## 3. Funciones solicitud datos

Función para obtener los datos del indicador *ind* solicitado entre dos fechas dadas.

En nuestro dataset se nos presentan 2 peculiaridades a introducir en la función:

1) Existe un único indicador para los precios de cada mercado, diferenciándose el pais por el parámetro GEOIDS --> **solicita_datos_precios**

2) Los datos de potencia disponible hay que agregarlos porque viene desglosada en comunidades

In [6]:
def solicita_datos_general(fecha_ini,fecha_fin,ind):

    headers_={'Accept': 'application/json; application/vnd.esios-api-v1+json',
              'Content-Type': 'application/json',
              'Host': 'api.esios.ree.es',
              'Authorization': 'Token token="9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10"',
              'Cookie':''
             }

    query_parametros_={'start_date': fecha_ini,
                'end_date':fecha_fin}

    url_='https://api.esios.ree.es/indicators/' + str(ind)
    
    #Hacemos peticion:
    r = requests.get(url_,headers=headers_,params=query_parametros_)
    
    #Comprobamos que trae datos
    assert r.status_code==200
    
    #r.content --> durante el diseño para explorar datos
    
    data=json.loads(r.content)
    
    #type(data)
    #data['indicator']['values'] --> durante el diseño para explorar datos
    
    df_data=pd.DataFrame(data['indicator']['values'])
    
    #df_data.dtypes --> Tenemos que convertir las fechas a formato fecha
    #y crear nuestro df
    
    df=pd.DataFrame()        
    df['datetime']=pd.to_datetime([f[:10] + ' ' + f[11:13] + ':00:00' for f in df_data['datetime']],yearfirst=True,format='%Y-%m-%d %H:%M:%S') 
    df['value']=df_data['value']
    if (ind==475) | (ind==476):
        df=df.groupby('datetime')['value'].sum()
        df=pd.DataFrame(df)
    else:
        df=df.set_index('datetime')

    return df

In [7]:
def solicita_datos_precios(fecha_ini,fecha_fin,ind,lista):
   
    headers_={'Accept': 'application/json; application/vnd.esios-api-v1+json',
              'Content-Type': 'application/json',
              'Host': 'api.esios.ree.es',
              'Authorization': 'Token token="9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10"',
              'Cookie':''
             }
 
    query_parametros_={'start_date': fecha_ini,
                       'end_date': fecha_fin}
 
    url_='https://api.esios.ree.es/indicators/'+ str(ind)
 
    #Hacemos peticion:
    r = requests.get(url_,headers=headers_,params=query_parametros_)
 
    #Comprobamos que trae datos
    assert r.status_code==200
 
    data=json.loads(r.content)
 
    data['indicator']['values']
 
    df_data=pd.DataFrame(data['indicator']['values'])
 
    df=pd.DataFrame()
    for i in enumerate(lista):
        df_subset=pd.DataFrame()
        df_data_subset=df_data[df_data['geo_id']==i[1][1]]
        df_data_subset.reset_index(drop=True,inplace=True)
        df_subset['datetime']=pd.to_datetime([f[:10] + ' ' + f[11:13] + ':00:00' for f in df_data_subset['datetime']],yearfirst=True,format='%Y-%m-%d %H:%M:%S')
        df_subset[str(i[1][0])]=df_data_subset['value']
        df_subset.set_index('datetime',inplace=True)
        df=pd.concat([df,df_subset],axis=1)
    return df      
 

In [8]:
pd.to_datetime?

Signature: pd.to_datetime(arg, errors='raise', dayfirst=False, yearfirst=False, utc=None, box=True, format=None, exact=True, unit=None, infer_datetime_format=False, origin='unix', cache=False)
Docstring:
Convert argument to datetime.

Parameters
----------
arg : integer, float, string, datetime, list, tuple, 1-d array, Series

    .. versionadded:: 0.18.1

       or DataFrame/dict-like

errors : {'ignore', 'raise', 'coerce'}, default 'raise'

    - If 'raise', then invalid parsing will raise an exception
    - If 'coerce', then invalid parsing will be set as NaT
    - If 'ignore', then invalid parsing will return the input
dayfirst : boolean, default False
    Specify a date parse order if `arg` is str or its list-likes.
    If True, parses dates with the day first, eg 10/11/12 is parsed as
    2012-11-10.
    with day first (this is a known bug, based on dateutil behavior).
yearfirst : boolean, default False
    Specify a date parse order if `arg` is str or its list-likes.

    - If T

## 4. Agregación dataset

Definimos parámetros de entrada:

La idea inicial es que mi dataset empiece en 01/10/2015 hasta 30/09/2018.

He escogido 3 años móviles, teniendo en cuenta que el año hidráulico comienza en octubre (y la hidraulicidad es importante en el Mercado Eléctrico) e intentar evitar el efecto de cambios regulatorios en la mayor medida posible

In [9]:
fecha_ini_='2015-09-30 00:00:00'  #para algunos datos nos interesa lo que ocurre el D-1. 
fecha_fin_='2018-09-30 23:59:29'

In [10]:
#Como son muchos datos y en ocasiones la API da error al pedirlos "del tirón", dividimos en 3:

chunks=[['2015-09-30 00:00:00','2016-09-30 23:59:59'],['2016-10-01 00:00:00','2017-09-30 23:59:59'],['2017-10-01 00:00:00','2018-09-30 23:59:59']]

In [11]:
datos_esios=pd.DataFrame()  #será nuestro set de datos
for i,j in chunks:
    datos_esios_sub=pd.DataFrame()
    for k,v in d.items():
        if (k==600) | (k==612) | (k==613):
            historico=solicita_datos_precios(i,j,k,v)
        else:    
            historico=solicita_datos_general(i,j,k)
            historico.columns=[str(v)]
        datos_esios_sub=pd.concat([datos_esios_sub,historico],axis=1)
    datos_esios=pd.concat([datos_esios,datos_esios_sub])

In [12]:
#Recopilo datos de P48_saldo interconexión
historico_saldo_interc=pd.DataFrame()
k='10015'
for i,j in chunks:
    historico_saldo_interc_sub=solicita_datos_general(i,j,k)
    historico_saldo_interc_sub.columns=['P48_SALDO_INTERC']
    historico_saldo_interc=pd.concat([historico_saldo_interc,historico_saldo_interc_sub])

Junto ambos dataframes con merge:

In [13]:
datos_esios_total=pd.merge(datos_esios,historico_saldo_interc,how='outer',left_index=True,right_index=True)

## 5. Check datos

In [14]:
datos_esios_total.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,P48_SALDO_INTERC
count,26334.000000,26334.000000,26334.000000,26334.000000,26334.000000,26334.000000,26334.000000,26334.000000,26334.000000,26294.000000
mean,5476.457166,28670.343332,4413.073935,4421.872587,4189.759577,48.744521,42.101879,48.472329,48.480976,1314.903685
std,3185.592338,4616.953599,2217.645272,477.265635,471.819696,14.949498,20.791275,15.090665,15.106684,1681.116829
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.000000,0.000000,-3400.000000
25%,3015.400000,24731.325000,2318.000000,4126.000000,3797.900000,40.140000,29.740000,40.050000,40.100000,661.925000
50%,4820.250000,28920.100000,4637.050000,4483.200000,4148.100000,49.555000,39.170000,49.000000,48.960000,2000.000000
75%,7285.625000,32265.650000,6286.175000,4799.700000,4705.300000,59.450000,51.550000,58.900000,58.740000,2500.000000
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.490000,103.610000,3500.000000


Al hacer el merge con P48_SALDO_INTERC y tener duplicados (las 02:00:00 de los días de 25 horas), me rellena todas las combinaciones y se me duplican los datos. Compruebo que esta afirmación es correcta y elimino duplicados después:

In [15]:
datos_esios_total[datos_esios_total.duplicated()]

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,P48_SALDO_INTERC
datetime,,,,,,,,,,
2015-10-25 02:00:00,4646.8,20074.5,4968.5,9481.4,8270.6,38.13,25.07,35.58,34.30,2000.0
2015-10-25 02:00:00,4941.3,19699.0,4691.2,9481.4,8270.6,38.00,25.02,35.00,35.00,2000.0
2016-10-30 02:00:00,1754.7,20106.3,6062.6,6600.0,7146.6,49.75,47.93,47.69,48.08,2800.0
2016-10-30 02:00:00,1734.8,19736.5,5887.4,6600.0,7146.6,47.90,46.70,46.65,47.77,2800.0
2017-10-29 02:00:00,7358.7,20268.2,2267.7,6143.4,6455.8,41.15,15.41,37.66,33.76,2800.0
2017-10-29 02:00:00,7145.0,19681.9,2247.9,6143.4,6455.8,40.90,25.79,36.86,25.00,2800.0


In [16]:
datos_esios_total.drop_duplicates(inplace=True)
datos_esios_total.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,P48_SALDO_INTERC
count,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26288.000000
mean,5476.657615,28672.335722,4413.087348,4421.192005,4189.052822,48.745913,42.104412,48.474281,48.483520,1314.625590
std,3185.745755,4615.591310,2217.773186,474.661673,469.408867,14.950758,20.792168,15.091625,15.106957,1681.198225
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.000000,0.000000,-3400.000000
25%,3015.600000,24733.475000,2318.000000,4125.800000,3797.900000,40.140000,29.747500,40.050000,40.100000,660.900000
50%,4820.250000,28921.700000,4636.800000,4483.200000,4148.100000,49.565000,39.170000,49.000000,48.970000,2000.000000
75%,7285.800000,32265.825000,6286.600000,4799.700000,4705.300000,59.450000,51.560000,58.900000,58.742500,2500.000000
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.490000,103.610000,3500.000000


En la columna de la interconexión tengo NaN (los datos que faltaban y me daban problemas en mi función).

Relleno los NaN como 0 (sin uso de la interconexión)

In [17]:
datos_esios_total[datos_esios_total['P48_SALDO_INTERC'].isna()==True]

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,P48_SALDO_INTERC
datetime,,,,,,,,,,
2016-05-02 07:00:00,7589.0,23413.4,968.5,3456.3,4003.4,20.00,36.17,15.00,12.80,NaN
2016-05-02 08:00:00,7068.3,25554.9,990.5,3456.3,4354.3,25.00,34.96,20.00,19.00,NaN
2016-05-02 09:00:00,6472.6,26855.5,910.5,3456.3,4354.3,27.19,31.69,24.09,21.19,NaN
2016-05-02 19:00:00,6780.3,26245.9,980.0,3456.3,4354.3,28.84,35.79,25.45,23.90,NaN
2016-05-02 22:00:00,8656.4,28485.3,1158.0,3456.3,4354.3,28.84,31.93,27.34,31.36,NaN
2016-05-03 06:00:00,8552.6,24201.0,1218.7,3456.3,4354.3,18.19,30.39,15.59,18.19,NaN
2016-05-03 07:00:00,8355.8,26894.9,1088.6,3456.3,4354.3,26.19,33.90,25.00,26.19,NaN
2016-05-03 08:00:00,8021.0,29496.6,1207.0,3456.3,4354.3,30.01,36.48,30.01,30.55,NaN
2016-05-03 09:00:00,7515.2,30602.2,1179.0,3456.3,4354.3,27.90,35.95,30.27,30.55,NaN


In [18]:
datos_esios_total['P48_SALDO_INTERC'].fillna(value=0,inplace=True)
datos_esios_total.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,P48_SALDO_INTERC
count,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000
mean,5476.657615,28672.335722,4413.087348,4421.192005,4189.052822,48.745913,42.104412,48.474281,48.483520,1312.628285
std,3185.745755,4615.591310,2217.773186,474.661673,469.408867,14.950758,20.792168,15.091625,15.106957,1680.700731
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.000000,0.000000,-3400.000000
25%,3015.600000,24733.475000,2318.000000,4125.800000,3797.900000,40.140000,29.747500,40.050000,40.100000,648.925000
50%,4820.250000,28921.700000,4636.800000,4483.200000,4148.100000,49.565000,39.170000,49.000000,48.970000,2000.000000
75%,7285.800000,32265.825000,6286.600000,4799.700000,4705.300000,59.450000,51.560000,58.900000,58.742500,2500.000000
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.490000,103.610000,3500.000000


In [19]:
datos_esios_total[datos_esios['PRECIO_MD_ESP']==datos_esios_total['PRECIO_I1']].count()

P48_EOLICA            3498
P48_DEMANDA           3498
P48_CARBON            3498
POT_DISP_HULLA_ANT    3498
POT_DISP_HULA_SB      3498
PRECIO_MD_ESP         3498
PRECIO_MD_FR          3498
PRECIO_I1             3498
PRECIO_I2             3498
P48_SALDO_INTERC      3498
dtype: int64

In [20]:
datos_esios_total[(datos_esios_total['PRECIO_MD_ESP']==datos_esios_total['PRECIO_I1']) & (datos_esios_total['PRECIO_I2']==datos_esios_total['PRECIO_I1'])].count()

P48_EOLICA            794
P48_DEMANDA           794
P48_CARBON            794
POT_DISP_HULLA_ANT    794
POT_DISP_HULA_SB      794
PRECIO_MD_ESP         794
PRECIO_MD_FR          794
PRECIO_I1             794
PRECIO_I2             794
P48_SALDO_INTERC      794
dtype: int64

De los 26.328 datos, hay 3.498 horas en los que el precio del mercado diario coincidió con el del intra, y de ellas, 794 horas en las que además el intradiario 2 tampoco subió/bajó con respecto al diario.

Nuestra clasificación será:

* Si PMI1 > PMD -> 0

* Si PMI1 < PMD -> 1

* Si PMI1 == PMD:

    * Si PMI2 > PMD -> 0
    * Si PMI2 <= PMD -> 1 

## 6. Salvar datos

In [21]:
datos_esios_total.to_csv('../data/datos_esios.csv',sep=';')